In [1]:
import pymongo
from pymongo import MongoClient
import os

In [2]:
import os
from dotenv import load_dotenv

# This will load the key-value pairs from your .env file into the environment
# It looks for the .env file in the current directory by default
load_dotenv(dotenv_path='./laws/.env')

# Now you can correctly access the variable by its name, which is 'MONGO_URI'
MONGO_URI = os.environ.get('mongo_uri')

# It's a good practice to check if the variable was loaded successfully
if MONGO_URI is None:
    print("Error: MONGO_URI environment variable not found. Please check your .env file.")
else:
    print("MONGO_URI loaded successfully!")

MONGO_URI loaded successfully!


In [3]:
# The name of your database and collection
DB_NAME = "legal_db"
COLLECTION_NAME = "legal_documents"

# The schema validation rules as a Python dictionary
# This is your final schema for Benin and Madagascar laws
schema_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["titre", "pays", "categorie", "langue", "contenu", "date_publication", "url_source", "vecteur_embedding", "mots_cles"],
        "properties": {
            "titre": {
                "bsonType": "string",
                "description": "doit être une chaîne de caractères et est requis"
            },
            "pays": {
                "enum": ["Bénin", "Madagascar"],
                "description": "doit être 'Bénin' ou 'Madagascar'"
            },
            "categorie": {
                "enum": ["Statut", "Jurisprudence", "Guide Juridique", "Texte Constitutionnel", "Décret", "Ordonnance", "Code des personnes et de la famille"],
                "description": "doit être l'une des catégories spécifiées"
            },
            "langue": {
                "enum": ["Français", "Malgache"],
                "description": "doit être 'Français' ou 'Malgache'"
            },
            "contenu": {
                "bsonType": "string",
                "description": "doit être une chaîne de caractères et est requis"
            },
            "date_publication": {
                "bsonType": "date",
                "description": "doit être une date et est requis"
            },
            "url_source": {
                "bsonType": "string",
                "description": "doit être une chaîne de caractères et est requis"
            },
            "vecteur_embedding": {
                "bsonType": "array",
                "description": "doit être un tableau de vecteurs et est requis"
            },
            "mots_cles": {
                "bsonType": "array",
                "items": {
                    "bsonType": "string"
                },
                "description": "doit être un tableau de chaînes de caractères"
            }
        }
    }
}

try:
    # Connect to MongoDB Atlas
    client = MongoClient(MONGO_URI)
    
    # Select the database and collection
    db = client[DB_NAME]
    
    # Execute the collMod command to set the validator
    db.command("collMod", COLLECTION_NAME, validator=schema_validator, validationAction="error")
    
    print(f"Schema validation successfully applied to the '{COLLECTION_NAME}' collection.")

except pymongo.errors.OperationFailure as e:
    # This will catch errors if the command fails (e.g., if the collection doesn't exist)
    print(f"Failed to apply schema validation: {e}")

finally:
    # Close the connection
    if 'client' in locals():
        client.close()

Schema validation successfully applied to the 'legal_documents' collection.
